# CNN 실습

## 라이브러리 호출

In [11]:
import tensorflow as tf
from tensorflow import keras

from keras.layers import Input, Dense, Conv2D, MaxPool2D, BatchNormalization, Dropout, Flatten
from keras.models import Model
from keras.backend import clear_session
from keras.callbacks import EarlyStopping

import numpy as np

## 데이터 불러오기

In [2]:
(x_train, y_train), (x_val, y_val) = tf.keras.datasets.mnist.load_data()

11501568/11490434 [==============================] - 0s 0us/step


In [3]:
print(x_train.shape , y_train.shape)
print(x_val.shape , y_val.shape)

(60000, 28, 28) (60000,)
(10000, 28, 28) (10000,)


## 데이터 전처리

### shape 명시

In [4]:
x_train = x_train.reshape(-1,28,28,1)

In [5]:
print(x_train.shape)

(60000, 28, 28, 1)


In [6]:
x_val = x_val.reshape(-1,28,28,1)

In [7]:
print(x_train.shape)

(60000, 28, 28, 1)


### scailing

In [8]:
max_num,min_num = x_train.max() , x_train.min()

x_train = (x_train - min_num) / (max_num - min_num)
x_val = (x_val - min_num) / (max_num - min_num)

### one hot encoding

In [9]:
from tensorflow.keras.utils import to_categorical

n_class = len(np.unique(y_train))

y_train = to_categorical(y_train,n_class)
y_val = to_categorical(y_val,n_class)


# 모델링

In [14]:
il = Input((28,28,1))
hl = Conv2D(filters=64,kernel_size=(3,3),padding='same',activation='relu')(il)
bl = BatchNormalization()(hl)
hl = MaxPool2D(pool_size=(2,2))(bl)
hl = Conv2D(filters=64,kernel_size=(3,3),padding='same',activation='relu')(hl)
hl = MaxPool2D(pool_size=(2,2))(hl)
bl = BatchNormalization()(hl)
fl = Flatten()(bl)
dl = Dropout(0.25)(fl)
hl = Dense(128,'relu')(dl)
ol = Dense(n_class,'softmax')(hl)

model = Model(il,ol)

model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer='adam',
              metrics=['accuracy'])

model.summary()


Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_4 (InputLayer)        [(None, 28, 28, 1)]       0         
                                                                 
 conv2d_6 (Conv2D)           (None, 28, 28, 64)        640       
                                                                 
 batch_normalization_6 (Batc  (None, 28, 28, 64)       256       
 hNormalization)                                                 
                                                                 
 max_pooling2d_6 (MaxPooling  (None, 14, 14, 64)       0         
 2D)                                                             
                                                                 
 conv2d_7 (Conv2D)           (None, 14, 14, 64)        36928     
                                                                 
 max_pooling2d_7 (MaxPooling  (None, 7, 7, 64)         0   

In [16]:
es = EarlyStopping(patience=5,
                   verbose=1,
                   restore_best_weights=True)

In [17]:
model.fit(x_train,y_train,128,10,3,[es],0.2,verbose=1)

Epoch 1/10
Epoch 2/10
Epoch 3/10
Epoch 4/10
Epoch 5/10
Epoch 6/10
Epoch 7/10
Epoch 8/10
Epoch 9/10
Restoring model weights from the end of the best epoch: 4.
Epoch 9: early stopping


In [20]:
pred = model.predict(x_val)

pred_y = pred.argmax(axis=1)
real_y = y_val.argmax(axis=1)


In [21]:
from sklearn.metrics import classification_report

print(classification_report(real_y,pred_y))

              precision    recall  f1-score   support

           0       0.99      0.99      0.99       980
           1       0.99      1.00      1.00      1135
           2       0.99      0.99      0.99      1032
           3       0.99      1.00      0.99      1010
           4       0.99      1.00      0.99       982
           5       0.99      0.99      0.99       892
           6       0.99      0.99      0.99       958
           7       0.99      0.99      0.99      1028
           8       0.99      0.99      0.99       974
           9       1.00      0.98      0.99      1009

    accuracy                           0.99     10000
   macro avg       0.99      0.99      0.99     10000
weighted avg       0.99      0.99      0.99     10000

